# Chapter 4. Dynamic Programming

 '동적 프로그래밍'이란 *MDP의 완벽한 환경 모델이 주어진 상태*에서 **최적의 정책을 계산하는 데에 사용될 수 있는 알고리즘의 집합**이라고 말할 수 있으며, '고전적인 동적 프로그래밍' 알고리즘들은 강화학습에서는 **perfect model 의 가정**과 **greate computational expense** 덕분에  제한된 도구들이지만, 여전히 이론적으로 중요한 위치를 차지하고 있으며 위의 2가지 가정을 제외하면 DP와 동일한 효과를 얻을 수 있다.
 
#### environment 는 대체적으로 finite MDP 를 가정한다
##### finite sets of state, action, and rewad
##### its dynamics are given by a set of probabilities $p(s',r\mid s,a)\ for\ all s\in S, a\in A(s), r\in R, and\ s'\in S^+$
#### continuous state and action space 경우에도 quantizing 을 통해 approximate solution을 찾는다
> DP의 핵심 아이디어는 (일반적인 강화학습에서도 그러하다) 좋은 정책들의 발견을 하기위한 가치함수의 사용에 있다.

Chapter 3 에서도 다루었지만 아래와 같이 최적의 가치함수를 얻을 수만 있다면 최적의 정책은 쉽게 얻을 수 있다.

$$
v_*(s) = E[R_{t+1} + \gamma max_{a'}q_*(S_{t+1}\mid S_t=s,A_t=a],\ (4.1)\\
q_*(s,a) = \begin{equation}
\sum_{s',r} p(s',r \mid s,a)[r+\gamma \max_{a'}q_*(s',a')]
\end{equation},\ (4.2)
$$
> DP 알고리즘들은 벨만 방정식을 위와 같이 원하는 가치함수들의 추정을 개선시킬 수 있는 규칙을 업데이트하는 할당 문제로 변경하여 얻을 수 있다.

### References.
1. [Examples of Policy Evaluation](https://www.puzzlr.org/policy-evaluation)
2. [Jack's Car Rental](https://github.com/ShangtongZhang/reinforcement-learning-an-introduction/blob/master/chapter04/CarRental.py)

## 4.1 Policy Evaluation (Prediction)
 임의의 정책$\pi$를 따르는 state-value function $v_\pi$ 를 계산하는 것을 DP 관점에서 보아 *policy evaluation*이라고 한다. 
 > 결국 우리가 원하는 것은 최적의 정책을 구하는 것인데 이를 모르는 상태에서 한 번 써봐야 하므로 '정책평가(policy evaluation)'라고 표현하는 것 같다.
 
$$
\begin{aligned}
v_\pi & \doteq E_\pi[G_t \mid S_t  = s]\ \\
& = E_\pi[R_{t+1} + \gamma G_{t+1} \mid S_t = s] &(from\ (3.9)) \\
& = E_\pi[R_{t+1} + \gamma v_\pi (S_{t+1}) \mid S_t = s] &(4.3) \\
& = \sum_a\pi(a \mid s) \sum_{s',r} p(s',r \mid s,a)[r + \gamma v_\pi(s')] &(4.4)
\end{aligned}
$$
> 임의의 정책 $\pi$를 따르고, 위의 수식에서 environment 의 dynamics 는 정확히 알고 있으며, $\gamma < 1$ 이라면 알려지지 않은 $(the\ v_\pi(s), s \in S)$를 구하는 것은 어렵지 않다.

$$
\begin{aligned}
v_{k+1}(s) & \doteq \sum_a\pi(a \mid s) \sum_{s',r} p(s',r \mid s,a)[r + \gamma v_k(s')]& (4.5)
\end{aligned}
$$
> 벨만 방정식에서 정책 부분은 같음을 보이고 있으며, $k \to \infty$ 인 경우 결국 수렴하게 되어 같은 정책을 가질 수 있으므로 이 알고리즘을 *반복적 정책 평가(iterative policy evaluation)* 이라고 한다.

##### 주어진 정책 하에서 이전의 값을 교체하는 과정을 *예측 업데이트(expected update)*라고 하며, 매 iteration 마다 '정책 평가' 과정에서 새로운 근사 가치 함수(approximation value function)을 생성하기 위하여 모든 상태의 값을 업데이트 한다.
##### 상태 혹은 상태-액션 쌍을 사용하는 것에 따라 '예측 갱신'을 구분하거나, 자식 상태의 예측 값들을 갱신하는 정확한 방법에 따라서도 구분 되어진다.
> DP 알고리즘에서 모든 상태가 업데이트 된 것을 *expected* update 라고 하는데 이는 sample data 가 아닌 모든 가능한 다음 상태에 대한 예측을 기반으로 하였기 때문이다.

#### 수식 4.5에서와 같이 상태를 업데이트 하기 위해서는 2개의 배열을 이용해서 계산이 가능하다
#### 또한 'in place' 업데이트를 통해 1개의 배열만으로도 계산할 수 있으며 값이 빨리 반영되어, 더 빨리 수렴한다.
![Iterative Policy](images/iterative_policy.png)

### Example 4.1: Gridworld (4x4)
![Consider the 4x4 gridworld](images/gridworld.png)
#### 종료 상태가 아닌 상태는 $S=\{1,2,...,14\}$ 14가지
#### 가능한 행동은 $A=\{up, down, right, left\}$ 4가지
#### 전이 확률은 100%이지만, 인접하지 않은 상태로는 0% ,격자 밖으로 이동 시에는 현재 상태 유지
#### 감가율은 없고(undiscounted), 에피소드를 가지는 작업(episodic task)

## 4.2 Policy Improvement
 정책의 가치함수를 계산하는 목적은 더 나은 정책들을 찾는 것인데, 언제 정책을 변경할 것인지 혹은 그대로 선택할 것인지 모호하기 때문에, 현재 상태에서 특정 액션을 선택하고 현재 존재하는 정책을 따른다. 결국 아래의 큐 함수가 된다. 
 
 $$
 \begin{aligned}
 q_\pi(s,a) & \doteq E[R_{t+1} + \gamma v_\pi(S_{t+1}) \mid S_t=s, A_t=a] & (4.6) \\
 & = \sum_{s',r}p(s',r \mid s,a)[r + \gamma v_\pi(s')].
 \end{aligned}
 $$
> 여기서의 핵심은 큐함수의 값이 가치함수의 값보다 클 지 혹은 작을 지 여부이며, 만약에 크다면 모든 s 의 상태에서는 a 를 선택하는 것이 최적이라고 할 수 있다. 이렇게 개선되는 방향으로 정책이 결정되는 것을 '정책 개선 정리(policy improvement theorem)' 라고 한다.

$$
\begin{aligned}
& q_\pi(s,\pi'(s)) \geq v_\pi(s). & (4.7) \\
\end{aligned}
$$
#### 임의의 정책 $\pi$와 $\pi'$가 있을 때에 모든 상태 s에 대하여 4.7이 성립한다고 하면 
$$
\begin{aligned}
& v_{\pi'}(s,\pi'(s)) \geq v_\pi(s). & (4.8)
\end{aligned}
$$
#### 동일한 정책 $\pi'$를 따르는 큐함수의 경우도 더 좋은 결과를 예상할 수 있다
$$
\begin{equation}
\begin{aligned}
\pi'(s) & \doteq \operatorname*{arg\,max}_a q_\pi(s,a) \\
& = \operatorname*{arg\,max}_a E[R_{t+1} + \gamma v_\pi(S_{t+1}) \mid S_t=s,\ A_t=a] & (4.9) \\
& = \operatorname*{arg\,max}_a \sum_{s',r}p(s',r \mid s,a)[r + \gamma v_\pi(s')],
\end{aligned}
\end{equation}
$$
#### '정책 개선 정리'에 따라 항상 탐욕 정책을 따르는 새로운 정책을 결정하는 것을 '정책 개선(policy improvement)' 라고 한다
> 이 수식은 벨만 최적 방정식과 동일하며, '정책 개선' 과정은 반드시 최적 정책을 제공해준다. 여태까지는 deterministic 한 정책에 대해서만 논의 하였으나 확률적 정책(stochastic policy)도 쉽게 적용이 가능한데 Figure 1.2 예제 에서도 마지막 optimal policy 가 정해진 이후에도 tie brake 가 발생하고 이를 선택하는 과정에서 random policy (stochastic)로 선택할 수 있다.

### Figure 4.1 
![Convergence of iteraive policy](images/gridworld2.png)
#### 좌측은 무작위 정책에 대한 상태-가치 함수의 근사값들을 표현한 것이고
#### 탐욕 정책을 따르는 가치함수의 예측을 나타낸 것이며 3번재 iteration에서 optimal임을 알 수 있다
> -1.0 에서 상하 좌우 계산을 해보면  0.25 x (-1 + 0.9 x -1) x 3 + 0.25 x (-1 + 0.9 x 0)  = 0.25 x (-1.9 x 3 - 1) = -1.675 = -1.7

#### PseudoCode. Iterative Policy Evaluation, for estimating $V \approx v_\pi$

## 4.3 Policy Iteration
정책 $\pi$가 $v_\pi$을 통해 $\pi'$으로 개선되고 다시 $\pi''$으로 개선될 수 있는데 이를 순차적으로 진행할 수 있다.
#### 정책평가 $\to$ 정책개선 $\to$ 정책평가 $\to$ 정책개선
##### finite MDP 환경에서는 모든 action, state 가 알려져 있으므로 optimal policy, optimal value function 을 구할 수 있으므로 위의 평가 & 개선 단계를 거치면서 반드시 정책은 개선되어 진다.
> 이러한 방법을 통해 최적 정책을 찾는 것을 '정책 반복(policy iteration)'이라고 한다. 

![Complete Policy Evaluation](images/complete_policy.png)
##### 1. Initialization: 가치함수와 정책함수를 임의의 값으로 초기화 한다.
##### 2. Policy Evaluation: 이전 가치와 추측 가치의 차 $\Delta$가 $\theta$ 보다 작아질 때까지 현재의 가치함수를 업데이트 한다.
#####  3. Policy Improvement: 탐욕정책을 수행하면서 같은 상태에서 같은 액션을 취할 때 까지 정책함수를 업데이트 한다.

#### PseudoCode. Policy Iteration (using iterative policy evaluation) for estimating $\pi \approx \pi_*$

### Example 4.2: Jack's Car Rental
잭은 전국적으로 2군데의 자동차 렌탈 회사를 관리한다. 매일 일정 수의 고객이 각 지점에서 차를 임대한다.
#### 1. 해당 지점에 자동차가 있다면 차를 빌려주고 회사로부터  10달러 를 받게 되고, 없다면 놓치게 된다.
#### 2. 자동차는 반환되고 나야만 임대가 가능하므로 2군데의 지점간에 차량을 1대 당 2달러에 이동할 수 있다.
#### 3. 각 지점의 차량 요청과 반납은 '포아송 확률 변수'에 따라 요청 $\lambda$ 는 3, 4 반납은 3, 2 이다 (숫자가 n 이 나올 확률은  $\frac{\lambda^n}{n!}e^{-\lambda}$)
#### 4. 지점 당 최대 수용 가능한 자동차는 20대 (넘어서는 자동차는 본사로 넘어가고 문제에서는 제외한다.)
#### 5. 하루 밤에 지점 간에 옮길 수 있는 자동차는 최대 5대
#### 6. 감가율 $\gamma = 0.9$, continuing finite MDP 환경
#### 7. time step 은 일자, 상태는 일 별 지점 당 보유 차량 수, 액션은 두 지점간에 이동하는 차량의 순수한 수
<br>

### Figure 4.2
![Jack's Car Rental](images/jackscar.png)
> 잭의 자동차 렌탈 예제처럼 정책 이터레이션은 가끔씩 아주 짧은 이터레이션 안에 수렴하기도 한다.

#### PseudoCode. Value Iteration for estimating $\pi \approx \pi_*$

## 4.4 Value Iteration
 정책 이터레이션의 약점은 매 이터레이션에 정책평가가 필요하다는 것인데, 이는 상태 집합을 반복하여 sweep 해야 한다는 의미이다. 비용이 큰 정책평가를 정확하게 수렴해야만 하는가? 혹은 빨리 끝낼 수 없는가? Figure 4.1 에서 보듯이 초기의 3번의 evaluation 이후에는 의미가 없어 보인다.
 > 사실 policy iteration 단계에서 policy evaluation 은 policy iteration 의 수렴을 보장하면서도 여러가지 방법으로 잘라낼 수 있다
 
#### 1. policy evaluation 을 단 한 번의 sweep 만 수행한다. (= **value iteration**)
> policy improvement 와 truncated policy evaluation 단계를 통합해서 단순히 업데이트 한다. 기존에는 policy evaluation 이 수렴한 이후에 다시 policy improvement 단계를 거쳤지만 evaluation 시에 improvement 까지 동시에 수행한다

> value iteration 은 벨만 최적 방정식을 update rule 로 변환하는 방식으로 이해할 수도 있는데 모든 액션의 maximum 값을 취한다는 점을 제외하고는 수식이 일치한다.

> 그리고 함수의 backup diagrams 을 비교하는 것도 도움이 된다.
![Value Iteration](images/value_iter.png)

### how value iteration terminates?
> $v_*$에 수렴할 때까지 이터레이션을 돌기 보다는 충분히 작은 변화량을 가지는 시점에서 중단할 수 있다. value iteration 작업은 policy evaluation 과 policy improvement 작업을 아주 잘 통합하여 더 빨리 수렴할 수 있다.

$$
\begin{equation}
\begin{aligned}
v_{k+1}(s) & \doteq \max_a E[R_{t+1} + \gamma v_\pi(S_{t+1}) \mid S_t=s,\ A_t=a] \\
& = \max_a \sum_{s',r}p(s',r \mid s,a)[r + \gamma v_k(s')], & (4.10)
\end{aligned}
\end{equation}
$$

### Example 4.3: Gambler's Problem
 겜블러는 동전 뒤집기의 순서의 결과에 베팅을 할 수 있는 기회를 가졌다. 동전의 앞면들이 나오는 경우 해당 동전의 튕김에 건 돈 만큼 벌 수 있다. 만약 뒷면들이 나온다면 걸었던 돈을 잃게 된다. 
#### 겜블러가 목표 금액인 100달러를 벌게 되거나, 돈을 모두 잃게 되는 경우 게임이 종료된다
#### 매 동전을 튕길 때 마다 겜블러는 반드시 자신의 돈의 일부를 정수배의 달러를 걸어야만 한다
#### 상태 state, s $\in$ { 1, 2, .... , 99 } 으로 겜블러가 가진 자본금
#### 행동 actions a $\in$ { 0, 1, ... , min(s, 100 - s) } 으로 베팅 할 수 있는 금액
#### 보상 reward 은 겜블러가 목적을 달성하는 경우(+1)를 제외하고 모든 전이 상황에 0 (zero)이다
#### 감가율은 없고(undiscounted), 에피소드를 가지는 작업(episodic task)이며 finite MDP 환경이다
#### 가치함수 state-value function 은 each state 마다 승리할 확률이다
#### 정책 policy 는 자본금의 수준(level of capital)과 베팅 금액(stakes)간의 맵핑이다
> $p_h$를 동전의 앞면이 나올 확률 이라고 할 때, $p_h$를 안다면 value-iteration 으로 해결할 수 있다.

### Figure 4.3
![Gambler Problem](images/gambler.png)

## 4.5 Asynchronous Dynamic Programming
 DP의 가장 큰 단점은 MDP의 모든 상태 집합에 대한 연산을 수행해야 한다는 점인데, 대용량의 데이터인 경우는 거의 불가능에 가깝다.
 
### *Asynchronous DP algorithms* 의 특성
#### 반복적으로 상태집합을 업데이트 하지 않는다 (not in-place sweep update)
#### 순서에 상관 없이 상태가 업데이트 될 수 있다
#### 어떤 상태가 한 번 변화할 때 특정 상태의 값은 몇 번씩이나 변경될 수 있다
#### 올바르게 수렴하기 위해서는 모든 상태에 대해 값의 업데이트가 이루어져야만 한다.
#### 반면에 업데이트 해야 할 상태를 선택하는 데에 있어서의 유연성은 크다

> 비동기 가치 이터레이션은 한 번에 하나의 상태만 업데이트 하는 예제를 생각해 보면, $0 \leq \gamma < 1$인 경우 무수히 많은 반복을 거침으로써 점근적으로 $v_*$에 수렴하게 된다. (상태 시퀀스가 stochastic 이라도 괜찮다. ), (심지어 undiscounted episode 의 경우에도 순서대로 수행하지 않으면 수렴하지 않는 경우도 회피하기 어렵지 않다)

> 유사하게, 비동기 truncated policy evaluation 을 위해서, policy evaluation 과 value iteration 업데이트를 섞는 것도 가능하다. 

> 물론 sweep-less DP 과정이 계산량 자체를 줄일 수 있다는 의미는 아니며, policy improvement 진행 전에 아주 긴 sweep 때문에 학습 과정에 lock이 잡히는 것을 피할 수 있어 조금 더 효과적이라는 의미이다. 또한 value information 값들의 순서를 변경하여 좀 더 효과적으로 업데이트 할 수 있을 것이며, 심지어는 optimal behavior에 도움되지 않는 업데이트는 skip 할 수도 있다 (Chapter 8)

> 실시간 상호작용과 계산을 섞는 것도 쉬우며 주어진 MDP 문제를 풀기위해 MDP 를 실제로 경험하는 에이전트를 동시에 수행시킬 수도 있다.


## 4.6 Generalized Policy Iteration

## 4.7 Efficiency of Dynamic Programming

## 4.8 Summary